# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [21]:
from io import BytesIO
from datetime import datetime
import csv
from pathlib import Path

import requests
import pdfplumber

In [22]:
url = 'https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_11.6.2018.pdf'
req = requests.get(url)

In [23]:
pdf = pdfplumber.open(BytesIO(req.content))

In [24]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [25]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [26]:
lines

[['Statewide Report by County - November 6, 2018',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Libertarian',
  'Republican',
  'Democratic',
  'NPA/IND',
  'Constitution',
  'OTH',
  'Inactive',
  'Total Active'],
 ['Aurora', '1', '794', '739', '267', '', '4', '88', '1,805'],
 ['Beadle', '16', '4,327', '3,292', '2,153', '5', '39', '1,033', '9,832'],
 ['Bennett', '2', '686', '816', '338', '', '2', '211', '1,844'],
 ['Bon Homme', '11', '2,077', '1,240', '607', '2', '', '296', '3,937'],
 ['Brookings', '70', '7,717', '5,220', '5,328', '27', '22', '1,689', '18,384'],
 ['Brown', '62', '10,406', '8,234', '4,295', '7', '40', '2,358', '23,044'],
 ['Brule', '10', '1,462', '1,191', '568', '5', '2', '216', '3,238'],
 ['Buffalo', '1', '169', '627', '138', '', '1', '77', '936'],
 ['Butte', '31', '3,888', '972', '1,302', '9', '5', '954', '6,207'],
 ['Campbell', '', '908', '120', '84', '', '', '119', '1,112'],
 ['Charles Mix', '3', '2,375', '2,414', '889', '2', '', '410', '

In [27]:
# report date is in the first line
report_date = lines.pop(0)

In [28]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2018-11-06


In [29]:
# save the PDF
outpath = (Path('pdfs') / f'{report_date}.pdf').resolve()

with open(outpath, 'wb') as outfile:
    outfile.write(req.content)

print(f'Wrote {outpath}')

Wrote /Users/laurelwinchester/south-dakota-voter-registration-data/pdfs/2018-11-06.pdf


In [30]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [31]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [32]:
headers

['county',
 'libertarian',
 'republican',
 'democratic',
 'npa_ind',
 'constitution',
 'other',
 'inactive',
 'total active']

In [33]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [34]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [35]:
data[0]

{'county': 'Aurora',
 'libertarian': 1,
 'republican': 794,
 'democratic': 739,
 'npa_ind': 267,
 'constitution': '',
 'other': 4,
 'inactive': 88,
 'date': '2018-11-06'}

In [36]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [37]:
# write to file
filepath = (Path('data') / f'{report_date}.csv').resolve()

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: /Users/laurelwinchester/south-dakota-voter-registration-data/data/2018-11-06.csv


In [38]:
%run build_files.py

Party list: americans_elect, constitution, democratic, inactive, independent, libertarian, no_labels, npa, npa_ind, npa_ind_oth, other, reform, republican
--------------------
Wrote sd-voter-registration-data.csv
Wrote sd-voter-registration-data-simplified.csv
Wrote README.md
